In [1]:
suppressMessages(library(MuSiC))
suppressMessages(library(Seurat))
suppressMessages(library(fastSave))
suppressMessages(library(pbmcapply))
suppressMessages(library(SingleCellExperiment))

## 数据导入及bulk数据整理

In [2]:
PFC.dgedata_150=readRDS.lbzip2(file='~/ReCIDE/benchmark测试/cross_dataset_PFC/ref_data/ref_all.rdsFS',n.cores = 200)
EXP<- readRDS("~/ReCIDE/benchmark测试/cross_dataset_PFC/EXP_and_key/EXP_PFC.rds")

###########################

In [3]:
######构建bulk
bulk.mtx<-as.data.frame(EXP[[1]])
for (i in 2:length(EXP)) {
  bulk.mtx2<-as.data.frame(EXP[[i]])
  bulk.mtx<-cbind(bulk.mtx,bulk.mtx2)
  
}
bulk.mtx<-as.matrix(bulk.mtx)
colnames(bulk.mtx)<-names(EXP)


In [4]:
###########################
ref_seurat=PFC.dgedata_150
kk<-as.data.frame(ref_seurat@assays[["RNA"]]@counts)
kksum<-as.data.frame(apply(kk, 2, sum))
all(kksum>0)
kksum<-subset(kksum,kksum[,1]>0,)
kksum<-as.data.frame(kksum)
ref_seurat<-ref_seurat[,row.names(kksum)]
rm(kksum)
rm(kk)

gc()


Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 60.9 GiB”


[1] TRUE

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,8589874,458.8,15305721,817.5,15305721,817.5
Vcells,3422833566,26114.2,32209662296,245740.3,33551660815,255978.9


In [5]:
#单细胞参考集是sce的形式
sce <- SingleCellExperiment(as.matrix(ref_seurat@assays$RNA@counts),
                            colData=DataFrame(label=ref_seurat@meta.data),
                            rowData=DataFrame(length=row.names(ref_seurat)))
names(assays(sce))<-'counts'

#在music前要先确定bulk和sc是否只有一列，只有一列的话无法运行（包括sample和cluster数是否为1）
music_output = music_prop(bulk.mtx = bulk.mtx, sc.sce = sce, clusters = 'label.major.celltype',
                              samples = 'label.subject',verbose = F)

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 60.9 GiB”


In [6]:
saveRDS(music_output,file='/home/lmh/ReCIDE/benchmark测试/cross_dataset_PFC/music/music_tutorial/MuSiC_output.rds')

In [4]:
music_output=readRDS('/home/lmh/ReCIDE/benchmark测试/cross_dataset_PFC/music/music_tutorial/MuSiC_output.rds')
prd_df=as.data.frame(t(music_output$Est.prop.weighted))

In [5]:
key_df<- readRDS("~/ReCIDE/benchmark测试/cross_dataset_PFC/EXP_and_key/key_PFC.rds")


In [6]:
library(dplyr)

prd_df=prd_df[sort(row.names(prd_df)),sort(colnames(prd_df))]
key_df=key_df[sort(row.names(key_df)),sort(colnames(key_df))]

# prd_df=prd_df[,c('P1','P2','P3','P4','P5','P6','P7','P8')]
# key_df=key_df[,c('P1','P2','P3','P4','P5','P6','P7','P8')]

rmse_vec=c()
for (i in 1:ncol(key_df)) {
  rmse_vec[i]=ModelMetrics::rmse(prd_df[,i],key_df[,i])
}


In [7]:
mean(rmse_vec)

[1] 0.03905149

In [11]:
saveRDS(prd_df,file='/home/lmh/ReCIDE/benchmark测试/cross_dataset_PFC/music/music_tutorial/MuSiC_prd_df.rds')